<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.8 MB/s eta 0:00:00


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [4]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [5]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [7]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 2024

# Beskrivelse av datasett

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28976 entries, 2022-10-31 17:37:00 to 2023-06-14 01:02:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  28976 non-null  int64  
 1   EnzymeType_A2  28976 non-null  int64  
 2   EnzymeType_B   28976 non-null  int64  
 3   EnzymeType_C   28976 non-null  int64  
 4   EnzymeType_D   28976 non-null  int64  
 5   EnzymeType_E   28976 non-null  int64  
 6   RawMatPercent  28976 non-null  float64
 7   NIRfat         28976 non-null  float64
 8   NIRash         28976 non-null  float64
 9   NIRwater       28976 non-null  float64
 10  TT08           28976 non-null  float64
 11  TT20           28976 non-null  float64
 12  TT12           28976 non-null  float64
 13  Mw             455 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [13]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10466 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10466 non-null  int64  
 1   EnzymeType_A2  10466 non-null  int64  
 2   EnzymeType_B   10466 non-null  int64  
 3   EnzymeType_C   10466 non-null  int64  
 4   EnzymeType_D   10466 non-null  int64  
 5   EnzymeType_E   10466 non-null  int64  
 6   RawMatPercent  10466 non-null  float64
 7   NIRfat         10466 non-null  float64
 8   NIRash         10466 non-null  float64
 9   NIRwater       10466 non-null  float64
 10  TT08           10466 non-null  float64
 11  TT20           10466 non-null  float64
 12  TT12           10466 non-null  float64
 13  Mw             332 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [14]:
# Oppretter instans for lagring av resultater
mw_res = Resultat_dicts()

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Hele treningssett

In [24]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                31  34.07
E                 15  16.48
B                 14  15.38
C                 14  15.38
D                 13  14.29
A1                 4   4.40


In [23]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_splitt = 10
n_gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=mw_res.dict_res_1_hel_opt,
                             dict_res=mw_res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [25]:
# Antall runder til optimalisering
n_runder = 50


# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                       trening,
                                                       test_andel=ubenyttet_andel,
                                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # Parameter til kryssvalidering
  n_gjentagelser = 2

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  n_splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  # Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
  veiledet_modell_evaluering_1(dict_res_opt=mw_res.dict_res_1_hel_opt,
                               dict_res=mw_res.dict_res_1_hel,
                               treningssett_markert=treningssett_markert,
                               enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_gjentagelser=n_gjentagelser,
                               n_runder=n_runder,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

Lagring av resultater

In [ ]:
mw_resultater_1_hel = pd.DataFrame(mw_res.dict_res_1_hel)
mw_resultater_1_hel_opt = pd.DataFrame(mw_res.dict_res_1_hel_opt)

mw_resultater_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel.csv')
mw_resultater_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_opt.csv')

## Alternativ 2: Hele treningssett

In [20]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                25  27.78
B                 16  17.78
C                 15  16.67
E                 15  16.67
D                 13  14.44
A1                 6   6.67


In [18]:
# Parametervalg for kryssvalidering
n_splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=mw_res.dict_res_2_hel_opt,
                             dict_res=mw_res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [21]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  #Enzymfordeling
  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  n_splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_2(dict_res_opt=mw_res.dict_res_2_hel_opt,
                               dict_res=mw_res.dict_res_2_hel,
                               rå_data=rå_data,
                               treningssett_markert=treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_runder=n_runder,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

Enzymtyper Fordeling: Treningssett markert andel: 0.2
            Frekvens  Andel
EnzymeCode                 
A2                41  26.28
C                 30  19.23
D                 30  19.23
B                 27  17.31
E                 26  16.67
A1                 2   1.28
Enzymtyper Fordeling: Treningssett markert andel: 0.4
            Frekvens  Andel
EnzymeCode                 
A2                62  33.16
C                 31  16.58
D                 31  16.58
B                 29  15.51
E                 28  14.97
A1                 6   3.21
Enzymtyper Fordeling: Treningssett markert andel: 0.6
            Frekvens  Andel
EnzymeCode                 
A2                83  37.22
E                 36  16.14
C                 33  14.80
D                 32  14.35
B                 31  13.90
A1                 8   3.59
Enzymtyper Fordeling: Treningssett markert andel: 0.8
            Frekvens  Andel
EnzymeCode                 
A2               109  37.33
E                 48  16.44


Lagring av resultater

In [ ]:
mw_resultater_2_hel = pd.DataFrame(mw_res.dict_res_2_hel)
mw_resultater_2_hel_opt = pd.DataFrame(mw_res.dict_res_2_hel_opt)

mw_resultater_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel.csv')
mw_resultater_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_opt.csv')

## Designproduksjon

## Alternativ 1: Hele treningssett

In [23]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 15  22.39
B                 13  19.40
C                 13  19.40
D                 12  17.91
A2                10  14.93
A1                 4   5.97


In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_splitt = 10
n_gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=mw_res.dict_res_1_design_opt,
                             dict_res=mw_res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50


# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, _, _, _ = trening_testsett_oppdeling_enzym(rå_data,
                                                       trening,
                                                       test_andel=ubenyttet_andel,
                                                       random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert

  # Parameter til kryssvalidering
  n_gjentagelser = 2

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  n_splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  # Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
  veiledet_modell_evaluering_1(dict_res_opt=mw_res.dict_res_1_design_opt,
                               dict_res=mw_res.dict_res_1_design,
                               treningssett_markert=treningssett_markert,
                               enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_gjentagelser=n_gjentagelser,
                               n_runder=n_runder,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

Lagring av resultater

In [ ]:
mw_resultater_1_design = pd.DataFrame(mw_res.dict_res_1_design)
mw_resultater_1_design_opt = pd.DataFrame(mw_res.dict_res_1_design_opt)

mw_resultater_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design.csv')
mw_resultater_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_opt.csv')

## Alternativ 2: Hele treningssett

In [24]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
E                 15  22.73
B                 13  19.70
C                 13  19.70
D                 12  18.18
A2                 9  13.64
A1                 4   6.06


In [ ]:
# Parametervalg for kryssvalidering
n_splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=mw_res.dict_res_2_design_opt,
                             dict_res=mw_res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, _, _\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       test_ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert

  #Enzymfordeling
  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  n_splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_2(dict_res_opt=mw_res.dict_res_2_design_opt,
                               dict_res=mw_res.dict_res_2_design,
                               rå_data=rå_data,
                               treningssett_markert=treningssett_markert,
                               testsett=testsett,
                               n_splitt=n_splitt,
                               n_runder=n_runder,
                               test_andel=test_andel,
                               trenings_andel=trenings_andel,
                               random_seed=random_seed)

Enzymtyper Fordeling: Treningssett markert andel: 0.2
            Frekvens  Andel
EnzymeCode                 
A2                41  26.28
C                 30  19.23
D                 30  19.23
B                 27  17.31
E                 26  16.67
A1                 2   1.28
Enzymtyper Fordeling: Treningssett markert andel: 0.4
            Frekvens  Andel
EnzymeCode                 
A2                62  33.16
C                 31  16.58
D                 31  16.58
B                 29  15.51
E                 28  14.97
A1                 6   3.21
Enzymtyper Fordeling: Treningssett markert andel: 0.6
            Frekvens  Andel
EnzymeCode                 
A2                83  37.22
E                 36  16.14
C                 33  14.80
D                 32  14.35
B                 31  13.90
A1                 8   3.59
Enzymtyper Fordeling: Treningssett markert andel: 0.8
            Frekvens  Andel
EnzymeCode                 
A2               109  37.33
E                 48  16.44


Lagring av resultater

In [ ]:
mw_resultater_2_design = pd.DataFrame(mw_res.dict_res_2_design)
mw_resultater_2_design_opt = pd.DataFrame(mw_res.dict_res_2_design_opt)

mw_resultater_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design.csv')
mw_resultater_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_opt.csv')

# NIR målinger ekskludert

In [28]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Hele treningssett

In [29]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Antall runder til optimalisering
n_runder = 2

# Parametervalg for kryssvalidering
n_splitt = 2
n_gjentagelser = 2

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=mw_res.dict_res_1_hel_opt_NIR,
                             dict_res=mw_res.dict_res_1_hel_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
mw_resultater_1_hel_NIR = pd.DataFrame(mw_res.dict_res_1_hel_NIR)
mw_resultater_1_hel_opt_NIR = pd.DataFrame(mw_res.dict_res_1_hel_opt_NIR)

mw_resultater_1_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_NIR.csv')
mw_resultater_1_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_opt_NIR.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Parametervalg for kryssvalidering
n_splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=mw_res.dict_res_2_hel_opt_NIR,
                             dict_res=mw_res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
mw_resultater_2_hel_NIR = pd.DataFrame(mw_res.dict_res_2_hel_NIR)
mw_resultater_2_hel_opt_NIR = pd.DataFrame(mw_res.dict_res_2_hel_opt_NIR)

mw_resultater_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_NIR.csv')
mw_resultater_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_opt_NIR.csv')

## Designproduksjon

## Alternativ 1: Hele treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_splitt = 10
n_gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=mw_res.dict_res_1_design_opt_NIR,
                             dict_res=mw_res.dict_res_1_design_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
mw_resultater_1_design_NIR = pd.DataFrame(mw_res.dict_res_1_design_NIR)
mw_resultater_1_design_opt_NIR = pd.DataFrame(mw_res.dict_res_1_design_opt_NIR)

mw_resultater_1_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_NIR.csv')
mw_resultater_1_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_opt_NIR.csv')

## Alternativ 2: Hele treningssett

In [27]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Parametervalg for kryssvalidering
n_splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=mw_res.dict_res_2_design_opt_NIR,
                             dict_res=mw_res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

Resultater

In [ ]:
mw_resultater_2_design_NIR = pd.DataFrame(mw_res.dict_res_2_design_NIR)
mw_resultater_2_design_opt_NIR = pd.DataFrame(mw_res.dict_res_2_design_opt_NIR)

mw_resultater_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_NIR.csv')
mw_resultater_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_opt_NIR.csv')